# Supervised Training Template

This notebook outlines the steps required to train a convolutional neural network on the labelled MGCLS dataset.  It is not a complete solution; instead it provides guidance and placeholders for you to implement your own logic.  Follow the comments in each cell and fill in the `TODO` sections to build your own training pipeline.


## 1. Set up environment

Import the required libraries.  You may need to install some packages via pip if they are not already available in your environment.  Ensure you are using a GPU runtime if available.

In [ ]:
# TODO: import modules
import os
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader

# Import your own modules from the `src` folder (adjust sys.path as needed)
import sys
sys.path.append('..')  # adjust this path if running from a different directory
from src.dataset import RadioDataset, parse_coords_from_filename, match_labels
from src.model import build_model
from src.utils import compute_f1, compute_map


## 2. Define file paths

Specify the locations of your extracted data and labels.  Update these variables to point to the directories on your own system or Colab environment.

In [ ]:
# TODO: set these paths appropriately
data_root = '/content/data'   # directory containing 'typ' and 'exo' subdirectories
labels_csv = '/content/labels.csv'

# Define the number of classes (you must supply this based on your label set)
num_classes = 8  # example number; adjust to your dataset


## 3. Load and inspect the labels

Use pandas to read `labels.csv` and explore its columns.  Identify the coordinate columns (e.g. `ra`, `dec`) and the label columns (e.g. `label1`, `label2`, ...).  You will need this information when implementing the coordinate matching function.

In [ ]:
# Read the labels CSV
labels_df = pd.read_csv(labels_csv)
labels_df.head()


## 4. Implement coordinate parsing and label matching

The dataset module in `src/dataset.py` provides skeleton functions `parse_coords_from_filename()` and `match_labels()`.  You need to implement these functions so they correctly extract coordinates from image filenames and find the nearest label entry.  Test your implementation in this cell.  For example, pick a few filenames from the `typ` directory and check that the returned labels make sense.

In [ ]:
import re

def parse_coords_from_filename(filename):
    """
    Extract RA and Dec from MGCLS filename, e.g., 'J123456.78-321456.7.png'
    Returns: tuple of floats (ra_deg, dec_deg)
    """
    # Strip extension
    basename = filename.split('.')[0] if '.' in filename else filename
    match = re.match(r'J(\d{6}\.?\d*)([+-]\d{6}\.?\d*)', basename)
    if not match:
        raise ValueError(f"Filename format not recognized: {filename}")

    ra_str, dec_str = match.groups()
    ra_deg = hms_string_to_deg(ra_str)
    dec_deg = dms_string_to_deg(dec_str)
    return ra_deg, dec_deg

def hms_string_to_deg(hms_str):
    """Convert RA from hhmmss.ss string to degrees"""
    h = int(hms_str[0:2])
    m = int(hms_str[2:4])
    s = float(hms_str[4:])
    return (h + m / 60.0 + s / 3600.0) * 15.0

def dms_string_to_deg(dms_str):
    """Convert Dec from ddmmss.ss string to degrees"""
    sign = -1 if dms_str.startswith('-') else 1
    dms_str = dms_str[1:] if dms_str[0] in '+-' else dms_str
    d = int(dms_str[0:2])
    m = int(dms_str[2:4])
    s = float(dms_str[4:])
    return sign * (d + m / 60.0 + s / 3600.0)


In [ ]:
def match_labels(coords, labels_df, tol_arcsec=1.0):
    """
    Match image coords (RA, Dec) to a row in labels_df using a tolerance in arcseconds.
    Returns: list of class label strings
    """
    import numpy as np

    ra, dec = coords
    # Convert to numpy for vectorized diff
    ra_diff = np.abs(labels_df['ra'].values - ra)
    dec_diff = np.abs(labels_df['dec'].values - dec)

    # Angular tolerance in degrees
    tol_deg = tol_arcsec / 3600.0

    matches = (ra_diff < tol_deg) & (dec_diff < tol_deg)
    if not matches.any():
        return []

    matched_row = labels_df[matches].iloc[0]
    label_str = matched_row['labels']  # assumed comma-separated string
    return [l.strip() for l in label_str.split(',') if l.strip()]


In [ ]:
# Load labels.csv
labels_df = pd.read_csv('data/labels.csv')

# Test with a file from 'typ/'
filename = os.listdir('data/typ')[0]
coords = parse_coords_from_filename(filename)
labels = match_labels(coords, labels_df)

print("Filename:", filename)
print("Coordinates:", coords)
print("Labels:", labels)

## 5. Create the dataset and dataloaders

Here we instantiate the `RadioDataset` for the labelled images.  You may choose to combine the typical and exotic datasets or create separate datasets and use `ConcatDataset`.  Apply appropriate transformations (e.g. resizing, normalisation, augmentation).

In [ ]:
# TODO: define transformations and instantiate the dataset
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5])
])

# Instantiate dataset (you may need to implement label conversion to multi‑hot here)
dataset = RadioDataset(image_root=os.path.join(data_root, 'typ'), labels_csv=labels_csv, transform=transform, label_df=labels_df)

# include exotic images:
exo_dataset = RadioDataset(image_root=os.path.join(data_root, 'exo'), labels_csv=labels_csv, transform=transform, label_df=labels_df)
from torch.utils.data import ConcatDataset
dataset = ConcatDataset([dataset, exo_dataset])

# Split into training and validation sets
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


## 6. Build the model

Create a ResNet‑based classifier using the helper function `build_model()` in `src/model.py`.  Remember to pass `num_classes` equal to the total number of labels you have.  Move the model to GPU if available.

In [ ]:
# Instantiate the model
model = build_model(num_classes=num_classes, pretrained=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function and optimiser
criterion = torch.nn.BCEWithLogitsLoss()  # consider pos_weight for class imbalance
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


## 7. Training loop

Implement the training loop.  For each batch, convert the list of label strings into a multi‑hot tensor.  Compute the loss, backpropagate, and update the model weights.  At the end of each epoch, evaluate the model on the validation set and compute metrics such as precision, recall, F1 and mAP using functions from `src/utils.py`.  Save the best model checkpoint.

In [ ]:
# Pseudocode for training loop (fill in the missing parts)
num_epochs = 10
best_f1 = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        images = batch['image'].to(device)
        # TODO: convert batch['label'] (list of strings) into a multi‑hot tensor of shape (batch_size, num_classes)
        # labels_tensor = ...
        labels_tensor = torch.zeros(len(images), num_classes)  # placeholder

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels_tensor.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    train_loss = running_loss / len(train_dataset)
    
    # Validation
    model.eval()
    y_true = []
    y_scores = []
    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            # TODO: convert labels
            labels_tensor = torch.zeros(len(images), num_classes)  # placeholder
            outputs = model(images)
            y_true.append(labels_tensor.numpy())
            y_scores.append(torch.sigmoid(outputs).cpu().numpy())
    y_true_arr = np.concatenate(y_true, axis=0)
    y_scores_arr = np.concatenate(y_scores, axis=0)
    precision, recall, f1 = compute_f1(y_true_arr, y_scores_arr)
    mAP = compute_map(y_true_arr, y_scores_arr)
    print(f'Epoch {epoch+1}: loss={train_loss:.4f}, f1={f1:.4f}, mAP={mAP:.4f}')
    # TODO: save best model
    if f1 > best_f1:
        best_f1 = f1
        # torch.save(model.state_dict(), 'best_model.pth')
        print('New best model saved')


## 8. Save the model

After training, save your model checkpoint to disk.  You can use this checkpoint in the semi‑supervised phase.

In [ ]:
# saving the trained model
torch.save(model.state_dict(), 'resnet18_supervised.pth')
